In [2]:
import numpy as np
import pandas as pd

# Group the political parties 

Here we manually grouped the political partie based on litterature and internet search.

For K-means clustering of the parties based on their recommendations for each votation see the notebook "Analysis of the political groups Clustering".

The manual and machine learning clusterings of the parties produced the same right and left groups

In [6]:
#create political party groups
party=pd.read_excel('../Data/Votation/Partis politiques Suisse.xlsx')
Droite=list(party.Sigle[party.Classification=='Droite'])
Gauche=list(party.Sigle[party.Classification=='Gauche'])

In [7]:
party

,Parti,Sigle,Tendance,Classification
0,Parti Libéral Radical,PLR,Droite,Droite
1,Parti Démocratique Chrétien,PDC,Droite,Droite
2,Parti Socialiste,PS,Gauche,Gauche
3,Union Démocratique du Centre,UDC,Droite,Droite
4,Parti Evangélique,PEV,Centre - Gauche,Gauche
5,Parti Chrétien Social,PCS,Centre - Gauche,Gauche
6,Parti Vert'libéraux,PVL,Centre - Droite,Droite
7,Parti Bourgeois Démocratique,PBD,Centre - Droite,Droite
8,Parti du Travail,PST,Gauche,Gauche
9,Parti Ecologiste,PES,Centre - Gauche,Gauche


### Assign a political weight to each party

A political weight was assigned to each party to compute the overall recommendations of the right and the left parties.
The political weight was defined as the national percentage of vote that each party received at the national coucil elections in 2015.

Not all the parties were represented at the national elections, thus only the ones that received more than 1% of the votes (political weight >1) were kept for futher analysis.

By keeping 9 parties we were able to represent 95% of the votes at the National Council elections

In [5]:
# import the various indicator table to get information about the vote for each party at the last elections
var_indic=pd.read_csv('Tables/Demographic/Clean/various_indicators',index_col=0)


In [6]:
var_indic.loc[['Switzerland'],var_indic.columns[61:]]

,Percentage of vote (National Council elections) FDP_2015,Percentage of vote (National Council elections) CVP_2015,Percentage of vote (National Council elections) SP_2015,Percentage of vote (National Council elections) SVP_2015,Percentage of vote (National Council elections) EVP_2015,Percentage of vote (National Council elections) GLP_2015,Percentage of vote (National Council elections) BDP_2015,Percentage of vote (National Council elections) PdA_2015,Percentage of vote (National Council elections) GPS_2015,Percentage of vote (National Council elections) Small right-wing parties_2015,Criminal offences as ‰ of total population under the Swiss Criminal Code_2014,Criminal offences as ‰ of total population under the Narcotics Act (NarcA)_2014,Criminal offences as ‰ of total population under the Foreign Nationals Act (FNA)_2014
Switzerland,16.396724,11.645937,18.840952,29.385955,2.103602,4.625862,4.103751,0.855597,7.05706,2.634263,64.630203,9.949591,4.858205


In [7]:
# the names were not the same in french or english: make a dictionnary to retrived the french name
english_names={'UDC': 'SVP',
              'PS':'SP',
              'PLR':'FDP',
              'PDC':'CVP',
              'PBD':'BDP',
              'PES':'GPS',
              'PVL':'GLP',
              'PEV':'EVP',
              'PST':'PdA'}

In [8]:
# find the % of vote received at the last national election to ponderate each party
political_weight={}
for name in english_names.keys():
    # associate to the french name the percentage of vote received in the election
    political_weight[name]=var_indic['Percentage of vote (National Council elections) '+ english_names[name]+'_2015']['Switzerland']


In [9]:
sum(political_weight.values()) # we captured 95% of the choice of the people with this list of party
# we will only take these into account to compute the main recommandation of the right and left groups

95.015440901799991

### Create a datafram with the parties and their political weight

In [10]:
party.drop('Tendance',axis=1, inplace=True)


In [11]:
party.set_index(party.Sigle, inplace=True, drop=True)

In [ ]:
party['Percentage of vote at national council 2015']='N/A'
for part in political_weight.keys():
    party['Percentage of vote at national council 2015'][part]=political_weight[part]

In [13]:
# Save the table 
writer = pd.ExcelWriter('Tables/Votation/pol_parties.xlsx')
party.to_excel(writer,sheet_name = 'Sheet1') # table is stored in Sheet 1
writer.save()

In [14]:
party

,Parti,Sigle,Classification,Percentage of vote at national council 2015
Sigle,,,,
PLR,Parti Libéral Radical,PLR,Droite,16.3967
PDC,Parti Démocratique Chrétien,PDC,Droite,11.6459
PS,Parti Socialiste,PS,Gauche,18.841
UDC,Union Démocratique du Centre,UDC,Droite,29.386
PEV,Parti Evangélique,PEV,Gauche,2.1036
PCS,Parti Chrétien Social,PCS,Gauche,N/A
PVL,Parti Vert'libéraux,PVL,Droite,4.62586
PBD,Parti Bourgeois Démocratique,PBD,Droite,4.10375
PST,Parti du Travail,PST,Gauche,0.855597


# Compute the recommendations of the right and the left groups

Now that we have the parties, which group they belong to and their political weight, we can compute the main recommendation for the right and left groups for each votation.

A recommendation for "yes" was assigned +1, a recommendation for "no" was assigned -1. These grades were then weighted by the political weight (compute above) of the party emmiting this recommendation.

A weighed recommendation was then obtained, for each group (right and left): if this score is positive it means the group recommends to vote for "yes", if this score is negative it means the group recommend to vote for "no".

Both group can have the same recommendation for one votation.

In [16]:
# associate oui non liberté de vote to 1, -1, 0
recom={'oui':1, 'non':-1, 'liberté de vote':0}

In [ ]:
# dictionnary to put one dataframe per year, because the parties change each year
Recommandation_year={}

#dictionnaries to put the parties in favor
in_favor={}
#against
against={}
# keep the number of all votations
votations=[]
for year in range(2011,2017):
    year=str(year)
    data=pd.read_excel('Tables/Votation/recommandation_parti_votations_federales.xls',sheetname=year,header=6)
    # drop the empty rows
    data.dropna(axis=0, how='any', inplace=True) 
    # remove the 'No ' in the column names
    data.columns=[col[3:] for col in data.columns]
    for col in data.columns:
        votations.append(col)
        #find the parties that were in favor
        in_favor[col]=data.index[data[col]=='oui'].tolist()
        #against
        against[col]=data.index[data[col]=='non'].tolist()        
    Recommandation_year[year]=data

In [18]:
# count the occurence of oui and non for each votation for each group of party, ponderate by the weight of each party

Recommandations=pd.DataFrame(index=votations)
Recommandations['Right']=0
Recommandations['Left']=0

for year in range(2011,2016):
    year=str(year)
    
    # for each votation
    for col in Recommandation_year[year].columns: 
        # go through the parties
        for part in Recommandation_year[year].index:

            # if the party is one we are interested in (that represent 95% of the votes)
            if part in english_names.keys():  
                # left parties
                if part in Gauche: # add to the previous score, ponderated by political weight
                    Recommandations['Left'][col]=Recommandations['Left'][col]+recom[Recommandation_year[year][col][part]]*political_weight[part]
                # right parties
                if part in Droite:
                    Recommandations['Right'][col]=Recommandations['Right'][col]+recom[Recommandation_year[year][col][part]]*political_weight[part]

In [15]:
# keep only the sign : if positive assign a +1 (corresponding to a yes), if negative assign a -1 (no)
Recommandations['Right_bool']=np.sign(Recommandations['Right'])
Recommandations['Left_bool']=np.sign(Recommandations['Left'])

In [16]:
Recommandations

,Right,Left,Right_bool,Left_bool
554,-55,27,-1,1
555,-64,27,-1,1
556,55,-27,1,-1
557,-64,22,-1,1
558,64,22,1,1
559,-41,27,-1,1
560,28,-27,1,-1
561,-6,-27,-1,-1
562,-2,-15,-1,-1
563,-23,27,-1,1


In [8]:
# associate oui non liberté de vote to 1, -1, 0
recom2={1:'oui', -1:'non'}

In [ ]:
# add a column with the recommandations of the right and the left
for vot in Recommandations.index:
    Recommandations['Right'][vot]=recom2[Recommandations['Right_bool'][vot]]
    Recommandations['Left'][vot]=recom2[Recommandations['Left_bool'][vot]]

In [19]:
# find the parties in favor of yes and of no
Recommandations['for_yes']=in_favor.values()
Recommandations['for_no']=against.values()

In [33]:
Recommandations.to_csv('Tables/Votation/education_expense')

# Compute the outcomes of the votations (which group won)

The outcome of the votation is provided by www.admin.ch as the % of yes and no in each canton. This is not meaningful when the votations are pulled together, as a "yes" for one votation can reflect the similar political opinion as a "no" for another votation. To analyse the votation outcome of the whole set of votations, we need to combine the votation outcome with the political meaning of this votation.

The recommendations computed before allow to determine if a votation was more right or left handed.

The outcome provided as a percentage was tranlated into the victory of yes (+1) or no (-1) for each votation. This binomial outcome was then used to assess which group "won" in each canton for each votation. If a group won, it means that their recommendation was the outcome (people followed their recommandations).

Because the groups can recommend the same vote there were 4 cases for who won: Neither left nor right won (0), Right group only won (1), Left group only won (2) or both groups won (3).

We computed this outcome (Won) as long as the outcome of each group (won : 1 or lost : -1; Right_won and Left_won), and the percentage of vote that the winning outcome received (Percent_won).



In [20]:
votations_table=pd.read_excel('Tables/Votation/all_votations_goodindex.xlsx',sheetname='Sheet1',header=[0,1])

In [25]:
votations_table

554                                                                \
         Canton  Electeurs    Votants % Particip.        Oui        Non % Oui   
0            ZH    873'069    427'317       48.94    214'164    210'976  50.4   
1            BE    710'987    376'365       52.94    152'348    221'611  40.7   
2            LU    257'570    129'362       50.22     51'306     76'713  40.1   
3            UR     26'034     11'344       43.57      3'300      7'929  29.4   
4            SZ     97'849     50'166       51.27     14'572     35'429  29.1   
5            OW     25'117     13'548       53.94      3'724      9'512  28.1   
6            NW     30'265     15'908       52.56      4'885     10'759  31.2   
7            GL     25'991     10'837       41.70      3'266      7'554  30.2   
8            ZG     71'663     38'436       53.63     16'403     21'913  42.8   
9            FR    183'402     85'487       46.61     34'847     49'851  41.1   
10           SO    172'059     83'883       48.75     29'204     54'161  35.0   
11           BS    113'781     55'783       49.03     32'606     22'732  58.9   
12           BL    186'188     92'760       49.82     41'351     50'137  45.2   
13           SH     49'494     32'019       64.69     11'944     18'713  39.0   
14           AR     37'459     19'372       51.72      7'284     11'999  37.8   
15           AI     11'266      4'998       44.36      1'360      3'554  27.7   
16           SG    309'408    147'988       47.83     57'760     89'687  39.2   
17           GR    135'061     59'564       44.10     20'227     38'557  34.4   
18           AG    395'978    183'898       46.44     70'604    112'468  38.6   
19           TG    159'148     79'080       49.69     26'731     50'682  34.5   
20           TI    210'256     92'820       44.15     33'564     58'538  36.4   
21           VD    407'150    201'321       49.45    107'098     92'354  53.7   
22           VS    203'893    103'643       50.83     38'581     62'788  38.1   
23           NE    109'519     51'466       46.99     27'176     23'868  53.2   
24           GE    238'654    114'103       47.81     68'776     43'889  61.0   
25           JU     50'391     19'792       39.28     10'231      9'438  52.0   
26  Switzerland  5'091'652  2'501'260       49.12  1'083'312  1'395'812  43.7   

                  555             ...    603                604             \
   % Non       Canton  Electeurs  ...  % Oui % Non       Canton  Electeurs   
0   49.6           ZH    881'041  ...   63.1  36.9           ZH    914'320   
1   59.3           BE    717'009  ...   56.4  43.6           BE    731'290   
2   59.9           LU    261'034  ...   57.3  42.7           LU    272'852   
3   70.6           UR     26'106  ...   50.7  49.3           UR     26'462   
4   70.9           SZ     98'679  ...   53.7  46.3           SZ    102'726   
5   71.9           OW     25'299  ...   49.7  50.3           OW     26'277   
6   68.8           NW     30'447  ...   55.2  44.8           NW     30'919   
7   69.8           GL     26'133  ...   57.6  42.4           GL     26'307   
8   57.2           ZG     72'196  ...   61.7  38.3           ZG     75'207   
9   58.9           FR    186'331  ...   67.9  32.1           FR    197'271   
10  65.0           SO    173'673  ...   56.1  43.9           SO    177'994   
11  41.1           BS    113'973  ...   60.5  39.5           BS    114'236   
12  54.8           BL    186'651  ...   60.2  39.8           BL    188'237   
13  61.0           SH     49'926  ...   53.0  47.0           SH     51'295   
14  62.2           AR     37'720  ...   49.5  50.5           AR     38'651   
15  72.3           AI     11'402  ...   49.7  50.3           AI     11'683   
16  60.8           SG    311'973  ...   54.5  45.5           SG    318'925   
17  65.6           GR    135'498  ...   54.8  45.2           GR    137'587   
18  61.4           AG    400'215  ...   57.9  42.1           AG    416'963   
19  65.5           TG   

## Step 1: compute the various outcomes for switzerland (All cantons together)

In [ ]:
Outcome=pd.DataFrame(index=votations)
Outcome['bool']=0
Outcome['Outcome']=0
Outcome['Won']='None'
for votation in votations:
    won=0
    # si le oui a gagné:
    if float(votations_table[int(votation)]['% Oui'][26])>float(votations_table[int(votation)]['% Non'][26]):
        Outcome['bool'][votation]=1
    else:
        Outcome['bool'][votation]=-1
        
    Outcome['Outcome'][votation]=recom2[Outcome['bool'][votation]]
    
    if Outcome['bool'][votation]==Recommandations['Right_bool'][votation]:
        won=1
        Outcome['Won'][votation]='Right'
    if Outcome['bool'][votation]==Recommandations['Left_bool'][votation]:
        if won==1:
            Outcome['Won'][votation]='Both'
        else:
            Outcome['Won'][votation]='Left'

In [27]:
Outcome['Recom_left']=Recommandations['Left']
Outcome['Recom_right']=Recommandations['Right']
    

In [ ]:
Outcome_bool=pd.DataFrame(index=votations)
Outcome_bool['Outcome']=Outcome['bool']
Outcome_bool['Left_won']=0
Outcome_bool['Right_won']=0
Outcome['Left_won']='non'
Outcome['Right_won']='non'

for vot in votations:
    if Outcome['Recom_left'][vot]==Outcome['Outcome'][vot]:
        Outcome_bool['Left_won'][vot]=1
        Outcome['Left_won'][vot]='oui'
    if Outcome['Recom_right'][vot]==Outcome['Outcome'][vot]:
        Outcome_bool['Right_won'][vot]=1
        Outcome['Right_won'][vot]='oui'

In [29]:
del Outcome['bool']

In [30]:
Outcome_bool['Recom_right']=Recommandations['Right_bool']
Outcome_bool['Recom_left']=Recommandations['Left_bool']

In [31]:
Outcome

,Outcome,Won,Recom_left,Recom_right,Left_won,Right_won
554,non,Right,oui,non,non,oui
555,oui,Left,oui,non,oui,non
556,non,Left,non,oui,oui,non
557,non,Right,oui,non,non,oui
558,oui,Both,oui,oui,oui,oui
559,non,Right,oui,non,non,oui
560,non,Left,non,oui,oui,non
561,non,Both,non,non,oui,oui
562,non,Both,non,non,oui,oui
563,oui,Left,oui,non,oui,non


In [32]:
# define number for noone won, right or left won
dict_wontobool={'None': 0 , 'Right': 1, 'Left' : 2, 'Both' : 3}

In [33]:
# add the won column to the boolean table
Outcome_bool['Won']=[dict_wontobool[i] for i in Outcome.Won]

In [29]:
Outcome.to_csv('Tables/Votation/Votations_outcome_switzerland')
Outcome_bool.to_csv('Tables/Votation/Votations_outcome_bool_switzerlan')

## Step 2 : compute the outcomes for each canton

In [48]:
# for each canton, compute who won, the outcomes for the right and the left and the percentage that obtained the winning score
Total_Outcome={}
percent_won=[]
for cant in range(0,26):
    
    Outcome_bool=pd.DataFrame(index=votations)
    Outcome_bool['Outcome']=0
    Outcome_bool['Won']=0
    Outcome_bool['Right_won']=-1
    Outcome_bool['Left_won']=-1
    
    for votation in votations:
        won=0
        # if yes won:
        if float(votations_table[int(votation)]['% Oui'][cant])>float(votations_table[int(votation)]['% Non'][cant]):
            Outcome_bool['Outcome'][votation]=1
            # keep the percentage of people that voted for yes
            percent_won.append(votations_table[int(votation)]['% Oui'][cant])
        
        # if no won:
        else:
            Outcome_bool['Outcome'][votation]=-1
            percent_won.append(votations_table[int(votation)]['% Non'][cant])
        
        # if the right recomanded what won
        if Outcome_bool['Outcome'][votation]==Recommandations['Right_bool'][votation]:
            #register that the right won
            Outcome_bool['Right_won'][votation]=1
            Outcome_bool['Won'][votation]=1
            
            
        if Outcome_bool['Outcome'][votation]==Recommandations['Left_bool'][votation]:#if the left won
            Outcome_bool['Left_won'][votation]=1
            #if both won
            if Outcome_bool['Right_won'][votation]==1:
                Outcome_bool['Won'][votation]=3
            else:
                Outcome_bool['Won'][votation]=2#only the left won

    Outcome_bool['Recom_right']=Recommandations['Right_bool']
    Outcome_bool['Recom_left']=Recommandations['Left_bool']
    
    # put the outcome in tables 

    Total_Outcome[votations_table[int(votation)]['Canton'][cant]]=Outcome_bool

In [50]:
Total_Outcome['BE']

,Outcome,Won,Right_won,Left_won,Recom_right,Recom_left
554,-1,1,1,-1,-1,1
555,1,2,-1,1,-1,1
556,-1,2,-1,1,1,-1
557,-1,1,1,-1,-1,1
558,1,3,1,1,1,1
559,-1,1,1,-1,-1,1
560,-1,2,-1,1,1,-1
561,-1,3,1,1,-1,-1
562,-1,3,1,1,-1,-1
563,1,2,-1,1,-1,1


In [51]:
#Make a table with one line for each pair canton-votation

pds = []
for cant in votations_table[int(votation)]['Canton'].tolist()[0:26]:
    table = Total_Outcome[cant]
    Rows = pd.MultiIndex.from_product([cant, votations])
    pds.append(pd.DataFrame(table.as_matrix(), index=Rows))

result = pd.concat(pds, axis=0)

In [53]:
result.columns=Outcome_bool.columns

In [54]:
result['percent_won']=percent_won

In [56]:
result.to_csv('Tables/Votation/Results_votations')

In [55]:
result

Outcome  Won  Right_won  Left_won  Recom_right  Recom_left  \
ZH 554        1    2         -1         1           -1           1   
   555        1    2         -1         1           -1           1   
   556       -1    2         -1         1            1          -1   
   557       -1    1          1        -1           -1           1   
   558        1    3          1         1            1           1   
   559       -1    1          1        -1           -1           1   
   560       -1    2         -1         1            1          -1   
   561       -1    3          1         1           -1          -1   
   562       -1    3          1         1           -1          -1   
   563        1    2         -1         1           -1           1   
   564       -1    3          1         1           -1          -1   
   565       -1    1          1        -1           -1           1   
   566        1    3          1         1            1           1   
   567        1    2         -1         1           -1           1   
   568        1    2         -1         1           -1           1   
   569        1    2         -1         1           -1           1   
   570       -1    3          1         1           -1          -1   
   571        1    1          1        -1            1          -1   
   572       -1    1          1        -1           -1           1   
   573        1    3          1         1            1           1   
   574        1    1          1        -1            1          -1   
   575       -1    1          1        -1           -1           1   
   576       -1    3          1         1           -1          -1   
   577       -1    3          1         1           -1          -1   
   578        1    3          1         1            1           1   
   579       -1    3          1         1           -1          -1   
   580       -1    3          1         1           -1          -1   
   581        1    3          1         1            1           1   
   582        1    1          1        -1            1          -1   
   583       -1    1          1        -1           -1           1   
...         ...  ...        ...       ...          ...         ...   
JU 566        1    3          1         1            1           1   
   567        1    2         -1         1           -1           1   
   568        1    2         -1         1           -1           1   
   569        1    2         -1         1           -1           1   
   570       -1    3          1         1           -1          -1   
   571        1    1          1        -1            1          -1   
   572       -1    1          1        -1           -1           1   
   573        1    3          1         1            1           1   
   574       -1    2         -1         1            1          -1   
   575       -1    1          1        -1           -1           1   
   576       -1    3          1         1           -1          -1   
   577       -1    3          1         1           -1          -1   
   578        1    3          1         1            1           1   
   579       -1    3          1         1           -1          -1   
   580       -1    3          1         1           -1          -1   
   581        1    3          1         1            1           1   
   582        1    1          1        -1            1          -1   
   583       -1    1          1        -1           -1           1   
   584       -1    2         -1         1            1          -1   
   585       -1    2         -1         1            1          -1   
   586        1    2         -1         1           -1           1   
   587       -1    1          1        -1           -1           1   
   588       -1    3          1         1           -1          -1   
   589       -1    3          1         1           -1          -1   
   590       -1    2         -1         1            1          -1   
   591       -1   